In [1]:
import pandas as pd
from pandas import DataFrame

import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [2]:
bag_df = pd.read_csv('bag_words.csv', sep=';')

In [3]:
bag_df.columns = ['link', 'categoria', 'texto']

In [4]:
bag_df.head()

,link,categoria,texto
0,http://g1.globo.com/economia/agronegocios/agro...,agro,criação peixes cativeiro brasil expansão país ...
1,http://g1.globo.com/economia/agronegocios/noti...,agro,acordo ibge abate somou milhões cabeças maior ...
2,http://g1.globo.com/sp/piracicaba-regiao/notic...,agro,universidade paulo usp piracicabasp anunciou i...
3,http://g1.globo.com/economia/midia-e-marketing...,agro,mcdonalds saladas compostas vegetais orgânicos...
4,http://g1.globo.com/economia/agronegocios/noti...,agro,anúncio feito presidente instituto agronômico ...


In [5]:
bag_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706 entries, 0 to 705
Data columns (total 3 columns):
link         706 non-null object
categoria    706 non-null object
texto        695 non-null object
dtypes: object(3)
memory usage: 16.6+ KB


In [7]:
bag_df.describe()

,link,categoria,texto
count,706,706,695
unique,696,6,685
top,http://g1.globo.com/educacao/noticia/ninguem-s...,ciencia-e-saude,acidente teori zavascki veja sabe agora é boat...
freq,2,146,3


In [8]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None)
forest = RandomForestClassifier(n_estimators = 100) 
pipe = Pipeline([('vect', vectorizer), ('forest', forest)])

In [10]:
bag_df_clean = bag_df.dropna()
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(bag_df_clean.texto, bag_df_clean.categoria)

In [11]:
pipe.fit(X_train, Y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...mators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

In [12]:
pipe.score(X_test, Y_test)

0.75287356321839083

In [27]:
url = 'http://g1.globo.com/rs/rio-grande-do-sul/noticia/2017/01/nas-proximas-horas-ou-dias-teremos-novo-relator-da-lava-jato-diz-mello.html'
def predict(url):
    from G1_categorizer import pre_processor
    words = pre_processor(url)
    resp = pipe.predict([words])
    print resp
    probs = pipe.predict_proba([words])
    resp = zip(pipe.classes_, probs[0])
    resp.sort(key=lambda tup: tup[1], reverse=True)
    print resp
    
predict(url)

['politica']
[('politica', 0.91000000000000003), ('economia', 0.040000000000000001), ('agro', 0.029999999999999999), ('educacao', 0.01), ('tecnologia', 0.01), ('ciencia-e-saude', 0.0)]
